OPTIMIZE
--------------------------------------------------------------------------------
What it does:

When data is written to a Delta table, Databricks stores it in many small Parquet files (especially after streaming or incremental loads).
Too many small files = slow query performance.

The OPTIMIZE command compacts small files into bigger, well-organized ones.
--------------------------------------------------------------------------------
Why it’s important?

Reduces the number of files → faster reads

Makes queries more efficient

Helps Z-Ordering perform better later
-------------------------------------------------------------------------------
When to use?

After large inserts or merges

At the end of your daily ETL pipeline

Before Z-Ordering

In [0]:
spark.sql("USE CATALOG safe_sure_catalog")
spark.sql("USE SCHEMA gold")

spark.sql("OPTIMIZE gold.transaction")




knowing *what happens* when you run `OPTIMIZE` helps you understand what’s going on “under the hood.”

Let’s break it down 👇

---

### 🧱 What Happens When You Run:

```sql
OPTIMIZE gold.transaction;
```

1️⃣ **Delta Lake scans your table**

* It looks inside the Delta log (`_delta_log` folder) to find all Parquet files that belong to `gold.transaction`.

2️⃣ **It identifies many small files**

* Especially after streaming or incremental writes, Delta creates hundreds of small files (like 1–5 MB each).
* Too many small files = slow queries, because Spark has to open each one.

3️⃣ **It merges them into larger Parquet files**

* `OPTIMIZE` groups small files together into bigger ones (100–1000 MB each).
* These larger files are faster to read because Spark can scan fewer files.

4️⃣ **It rewrites the Delta log**

* The `_delta_log` is updated to point to the new compacted files.
* The old files are marked as obsolete (they stay temporarily for time travel).

---

### 🧾 How You’ll Know It Worked

After running `OPTIMIZE`, Databricks shows a **result table** like this:

| numFilesAdded | numFilesRemoved | totalFilesSizeRemoved | totalFilesSizeAdded | totalBytesRemoved | totalBytesAdded | numOptimizedFiles | durationMs |
| ------------- | --------------- | --------------------- | ------------------- | ----------------- | --------------- | ----------------- | ---------- |
| 15            | 200             | 1.2 GB                | 400 MB              | 1,200,000,000     | 400,000,000     | 185               | 12,345     |

✅ This means:

* It **merged 200 small files into 15 bigger files**.
* It **freed up space** (or reorganized it).
* The operation took a few seconds to minutes depending on data size.

---

### 🧠 How to Verify It Worked

#### Option 1: Check file count before and after

```sql
DESCRIBE DETAIL gold.transaction;
```

Look at the **numFiles** and **sizeInBytes** fields — after optimization, `numFiles` should drop sharply.

#### Option 2: See history log

```sql
DESCRIBE HISTORY gold.transaction;
```

You’ll see an entry like:

```
operation = OPTIMIZE
operationParameters = {predicate: '[]'}
userName = your.name@databricks.com
```

#### Option 3: Measure performance

Run the same query **before and after** optimization — it should execute faster, especially for aggregations.

---

### ⚙️ When to Run in a Project

| Stage                           | When to Run `OPTIMIZE`   | Why                              |
| ------------------------------- | ------------------------ | -------------------------------- |
| 🏗️ After initial load          | After large batch insert | Consolidate first write          |
| 🔁 During pipeline runs         | Daily or weekly          | Clean up after incremental loads |
| 📊 Before performance testing   | Before heavy queries     | Improve read speed               |
| 🧼 After merging data (upserts) | After `MERGE INTO`       | Remove small delta fragments     |

---


ZORDER BY


What it does:
Z-Ordering is like sorting and clustering data in Delta files based on certain columns (e.g., date or customer).
This helps Databricks quickly find relevant data during queries instead of scanning the whole dataset.
-------------------------------------------------------------------------------------------------------
 Example:
OPTIMIZE gold.transaction
ZORDER BY (TransactionDate, CustomerID);
-------------------------------------------------------------------------------------------------------
Why it’s important:
•	Speeds up queries that filter or join on specific columns
•	Reduces scan time and cost
-------------------------------------------------------------------------------------------------------
⏰ When to use:
•	After running OPTIMIZE
•	On columns frequently used in WHERE, GROUP BY, or JOIN
________________________________________


In [0]:


spark.sql("OPTIMIZE gold.transaction ZORDER BY (TransactionDate, CustomerID)")

Describe

DESCRIBE HISTORY
🔹 What it does:
This shows the audit log (or “timeline”) of every change made to a Delta table — inserts, updates, deletes, optimizations, etc.
----------------------------------------------------------------------------------------------------------------

📈 Why it’s important:
•	Helps you track when data changed
•	Useful for debugging ETL jobs
•	Helps identify which version to restore or time travel to
----------------------------------------------------------------------------------------------------------------
⏰ When to use:
•	Before and after updates/deletes
•	Before VACUUM to check versions


In [0]:
df_history = spark.sql("DESCRIBE HISTORY gold.transaction")
display(df_history)


TIME TRAVEL

🔹 What it does:
Delta Lake automatically keeps older versions of your data.
You can query or restore them using version number or timestamp.
-----------------------------------------------------------------------------------------

📈 Why it’s important:
•	Lets you undo accidental changes
•	Allows data audits and reproducible analysis
•	Enables debugging ("What did the table look like last week?")
-----------------------------------------------------------------------------------------
⏰ When to use:
•	Before or after major updates
•	During audits or validation checks
________________________________________


In [0]:
%sql
SELECT * FROM gold.transaction VERSION AS OF 0;

In [0]:
%sql
-- Or by timestamp
SELECT * FROM gold.transaction TIMESTAMP AS OF '2025-11-19 14:00:07';

RESTORE


🔹 What it does:
This restores your Delta table to a specific previous version.
------------------------------------------------------------------------

📈 Why it’s important:
•	Quickly roll back to a good version if something goes wrong
•	Saves hours compared to reloading from scratch
-------------------------------------------------------------------------
⏰ When to use:
•	After a bad data load, delete, or corruption
________________________________________


In [0]:
%sql

RESTORE TABLE gold.transaction TO VERSION AS OF 1;

VACCUM

🔹 What it does:
Deletes old Parquet files that are no longer referenced by the Delta transaction log.
By default, Delta keeps old data files for 7 days for time travel.
------------------------------------------------------------------------------------
📈 Why it’s important:
•	Frees up storage space
•	Removes stale file references
•	Improves performance on large tables
-----------------------------------------------------------------------------------
⚠️ Caution:
Once you VACUUM, you can’t time travel to versions older than the retention period.
-----------------------------------------------------------------------------------
⏰ When to use:
•	After confirming your data is stable
•	In scheduled maintenance jobs (e.g., weekly or monthly)


In [0]:
%sql

-- Normal cleanup
VACUUM gold.transaction;

-- Aggressive cleanup (for testing)
VACUUM gold.transaction RETAIN 0 HOURS;